# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [56]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key 
from api_keys import geoapify_key

In [57]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,smithers,54.7804,-127.1743,-2.82,68,100,2.06,CA,1698144377
1,1,nemuro,43.3236,145.5750,11.40,72,87,5.25,JP,1698144378
2,2,klyuchi,52.2667,79.1667,5.30,47,100,8.13,RU,1698144379
3,3,hermanus,-34.4187,19.2345,17.53,71,32,4.12,ZA,1698144381
4,4,rio grande,-32.0350,-52.0986,21.06,97,67,5.20,BR,1698144383


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [58]:
%%capture --no-display

city_map = city_data_df.hvplot.points(x="Lng", 
                                      y="Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [59]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df["Max Temp"] < 27) &
                               (city_data_df["Max Temp"] > 21) &  
                               (city_data_df["Wind Speed"] < 4.) &  
                               (city_data_df["Cloudiness"] == 0)].copy()

# Drop any rows with null values
ideal_weather_df.dropna(inplace=True)

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
130,130,pokhara,28.2333,83.9833,21.68,58,0,0.32,NP,1698144483
196,196,cabo san lucas,22.8909,-109.9124,23.75,77,0,3.13,MX,1698144043
265,265,nelson bay,-32.7167,152.1500,23.03,91,0,3.13,AU,1698144769
267,267,groves,29.9483,-93.9171,23.03,94,0,2.57,US,1698144771
333,333,petit-bourg,16.1914,-61.5916,26.77,94,0,0.00,GP,1698144855


### Step 3: Create a new DataFrame called `hotel_df`.

In [60]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
130,pokhara,NP,28.2333,83.9833,58,
196,cabo san lucas,MX,22.8909,-109.9124,77,
265,nelson bay,AU,-32.7167,152.1500,91,
267,groves,US,29.9483,-93.9171,94,
333,petit-bourg,GP,16.1914,-61.5916,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [61]:
# Set parameters to search for a hotel
radius = 10000  # Radius in meters
params = {"type": "lodging",  # Type of place to search for (lodging for hotels)
          "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"radius(10000, {latitude},{longitude})"
    params["bias"] = f"proximity({latitude},{longitude})"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pokhara - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
nelson bay - nearest hotel: No hotel found
groves - nearest hotel: No hotel found
petit-bourg - nearest hotel: No hotel found
ebaye - nearest hotel: No hotel found
emirgazi - nearest hotel: No hotel found
gunnedah - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
130,pokhara,NP,28.2333,83.9833,58,No hotel found
196,cabo san lucas,MX,22.8909,-109.9124,77,No hotel found
265,nelson bay,AU,-32.7167,152.1500,91,No hotel found
267,groves,US,29.9483,-93.9171,94,No hotel found
333,petit-bourg,GP,16.1914,-61.5916,94,No hotel found
435,ebaye,MH,8.7748,167.7392,89,No hotel found
484,emirgazi,TR,37.9022,33.8372,31,No hotel found
500,gunnedah,AU,-30.9833,150.2500,16,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [62]:
%%capture --no-display

# Configure the map plot
city_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    color="City",
    alpha=0.6,
    size="Humidity",
    tiles="OSM",
    hover_cols=["Hotel Name", "Country" ]
)

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)